# <center>ディープラーニング入門</center>
# <center>A LSTM in Wonderland</center>


In [1]:

import os
os.environ["CUDA_VISIBLE_DEVICES"]="2"
import tensorflow as tf
physical_devices = tf.config.experimental.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(physical_devices[0], True)


In [2]:
import numpy as np

# 1. 文字単位での文章生成

今回はLSTMで文章生成してみます。Kerasのexampleを少し改造したモデルを用意しました。

## 1-1. テキストデータのクリーニング

青空文庫から不思議の国のアリスのテキストファイルをダウンロードしましょう。

https://www.aozora.gr.jp/cards/001393/card57320.html

解凍したファイルのうち、alicewa_fushigino_kunide.txtをGoogle Driveのaliceフォルダに入れておきます。

In [3]:
# file_path = './text/alicewa_fushigino_kunide.txt'
    
# with open(file_path, 'r', encoding='shift-jis') as text_file:
#     strings = text_file.read()
    
# paras = strings.splitlines()

開いたテキストを見てみます：

In [4]:
# !cp ../../dm_integration/data/EPG_checking\ 0212_check.xlsx ./
!ls

A_thousand_ball_drill1202-Copy1.ipynb  EPG.xlsx  text
A_thousand_ball_drill1202.ipynb        output


In [5]:
import pandas as pd

In [6]:
df = pd.read_excel("./EPG.xlsx")

In [ ]:
# paras

In [18]:
paras = list(df.iloc[:, 11])

In [19]:
# df.iloc[:, 11][1000]

In [20]:
len(paras)

6153

In [21]:
paras

['ついにバディ結成で最初の事件に挑む!新宿駅で死んだ女は別人に成り代わった女だった…。“ディーン"シャーロックが対峙するのは、美しき弁護士・菅野美穂!天才が唯一惚れた特別な聖女との息詰まる心理戦が開幕!オリンピックを翌年に控えたTOKYOの街を舞台に、現代の日本に生まれ変わった世界一有名なバディ〈シャーロック=誉獅子雄×ワトソン=若宮潤一〉が、数々の難事件の謎に立ち向かうミステリーエンタテインメント上の階からの水漏れにより宿無しとなった誉獅子雄(ディーン・フジオカ)は、医師の転落死を巡る事件で知り合った若宮潤一(岩田剛典)のマンションに転がり込んだ。そんな中、捜査一課の江藤礼二(佐々木蔵之介)から獅子雄のもとへ新たな捜査依頼が舞い込んでくる。それはとある女性が急行電車にはねられ死亡した一件。所有していた身分証から亡くなったのは29歳の高橋博美だと判明する。博美はかつて、とあるトラブルを抱えており、敏腕弁護士の青木藍子(菅野美穂)に弁護の依頼をしていた。藍子は、依頼を受けた人にはたとえ弁護が終わっても一生寄り添うことをポリシーとする弁護士であり、生きづらさを抱える人々に対しグループワークを開くなど、社会的弱者の側に立った活動を一貫して続けている人格者。そんな藍子と博美の関係を知った獅子雄は、調査目的で藍子のもとを訪ねる。そこで獅子雄は弁護士としても女性としても完璧な藍子に興味を抱き…。',
 'オリンピックを翌年に控えたTOKYOの街を舞台に、現代の日本に生まれ変わった世界一有名なバディ〈シャーロック=誉獅子雄×ワトソン=若宮潤一〉が、数々の難事件の謎に立ち向かうミステリーエンタテインメント!今回のテーマは、地面師詐欺!巨額不動産を巡り詐欺師との息詰まる騙し合いが展開!そして殺された詐欺師が残したダイイングメッセージを解け!現場に残された長・短・短の3本の枝の謎!終わらないどんでん返し!渋谷の一等地にある空き家で、身元不明の高齢男性の死体が発見された。そこは5年前に巨額の地面師詐欺の舞台になった邸宅。事件後、詐欺被害にあった建設会社の社長がその邸宅内で自殺し、いわく付き物件として空き家のままになっていた。  捜査一課の江藤礼二(佐々木蔵之介)に捜査依頼を受けた誉獅子雄(ディーン・フジオカ)は、その遺体があった場所を検分。すると、遺体の手元に3本の木の枝があることに気

色々と本文とは関係ないものが混ざっていますので、それらを取り除きます。こういうことは手作業でしません（自動化しにくく最後まで残ってしまうちょっとしたゴミだけ、手作業で除去しましょう）：

In [30]:
import re
import codecs

save_path = './output/alice.txt'
    
end = 0
# あとがき以下削除
# for i in range(150):
#     if paras[-i]=='子どものみなみなさま':
#         end=i+3
# paras = paras[:-end]

# # 本文以前の冒頭を削除     
# for i, p in enumerate(paras):
#     if p[:4]=='----':
#         end = i+3
# paras = paras[end:]

# # 見出し等削除して保存
r = codecs.open(save_path, 'w', 'utf8')
marks = re.compile(r'［[^［]*］|《[^《]*》|｜|\u3000')
for ind, p in enumerate(paras):
    if p =="":
        next
    q = re.sub(marks, '', p)
    r.write(q)
    
    if ind > 550:
        break
r.close()

In [ ]:
paras

特殊な挿入物は正規表現で削除していきます。

ではクリーニング後のテキストを確認してみましょう：

In [31]:
import io

with io.open(save_path, encoding='utf-8') as f:
    text = f.read()

print('コーパス長:', len(text))

コーパス長: 338590


In [32]:
text

'ついにバディ結成で最初の事件に挑む!新宿駅で死んだ女は別人に成り代わった女だった…。“ディーン"シャーロックが対峙するのは、美しき弁護士・菅野美穂!天才が唯一惚れた特別な聖女との息詰まる心理戦が開幕!オリンピックを翌年に控えたTOKYOの街を舞台に、現代の日本に生まれ変わった世界一有名なバディ〈シャーロック=誉獅子雄×ワトソン=若宮潤一〉が、数々の難事件の謎に立ち向かうミステリーエンタテインメント上の階からの水漏れにより宿無しとなった誉獅子雄(ディーン・フジオカ)は、医師の転落死を巡る事件で知り合った若宮潤一(岩田剛典)のマンションに転がり込んだ。そんな中、捜査一課の江藤礼二(佐々木蔵之介)から獅子雄のもとへ新たな捜査依頼が舞い込んでくる。それはとある女性が急行電車にはねられ死亡した一件。所有していた身分証から亡くなったのは29歳の高橋博美だと判明する。博美はかつて、とあるトラブルを抱えており、敏腕弁護士の青木藍子(菅野美穂)に弁護の依頼をしていた。藍子は、依頼を受けた人にはたとえ弁護が終わっても一生寄り添うことをポリシーとする弁護士であり、生きづらさを抱える人々に対しグループワークを開くなど、社会的弱者の側に立った活動を一貫して続けている人格者。そんな藍子と博美の関係を知った獅子雄は、調査目的で藍子のもとを訪ねる。そこで獅子雄は弁護士としても女性としても完璧な藍子に興味を抱き…。オリンピックを翌年に控えたTOKYOの街を舞台に、現代の日本に生まれ変わった世界一有名なバディ〈シャーロック=誉獅子雄×ワトソン=若宮潤一〉が、数々の難事件の謎に立ち向かうミステリーエンタテインメント!今回のテーマは、地面師詐欺!巨額不動産を巡り詐欺師との息詰まる騙し合いが展開!そして殺された詐欺師が残したダイイングメッセージを解け!現場に残された長・短・短の3本の枝の謎!終わらないどんでん返し!渋谷の一等地にある空き家で、身元不明の高齢男性の死体が発見された。そこは5年前に巨額の地面師詐欺の舞台になった邸宅。事件後、詐欺被害にあった建設会社の社長がその邸宅内で自殺し、いわく付き物件として空き家のままになっていた。  捜査一課の江藤礼二(佐々木蔵之介)に捜査依頼を受けた誉獅子雄(ディーン・フジオカ)は、その遺体があった場所を検分。すると、遺体の手元に3本の木の枝があることに気づく。江藤は

無事綺麗になりました。

## 1-2. 訓練データの準備

テキストに現れる固有の文字を、setにすることで数え上げてみましょう：

In [45]:
chars = sorted(list(set(text)))

print('全文字数:', len(chars))

全文字数: 2453


In [46]:
chars

[' ',
 '!',
 '"',
 '%',
 '&',
 '(',
 ')',
 '*',
 ',',
 '-',
 '.',
 '/',
 '0',
 '1',
 '2',
 '3',
 '4',
 '5',
 '6',
 '7',
 '8',
 '9',
 ':',
 '=',
 '?',
 '@',
 'A',
 'B',
 'C',
 'D',
 'E',
 'F',
 'G',
 'H',
 'I',
 'J',
 'K',
 'L',
 'M',
 'N',
 'O',
 'P',
 'Q',
 'R',
 'S',
 'T',
 'U',
 'V',
 'W',
 'X',
 'Y',
 'Z',
 '_',
 'a',
 'b',
 'c',
 'd',
 'e',
 'f',
 'g',
 'h',
 'i',
 'j',
 'k',
 'l',
 'm',
 'n',
 'o',
 'p',
 'r',
 's',
 't',
 'u',
 'v',
 'w',
 'x',
 'y',
 'z',
 '×',
 '“',
 '…',
 '※',
 '→',
 '─',
 '◆',
 '◇',
 '★',
 '☆',
 '、',
 '。',
 '々',
 '〈',
 '〉',
 '「',
 '」',
 '『',
 '』',
 '【',
 '】',
 'ぁ',
 'あ',
 'い',
 'ぅ',
 'う',
 'ぇ',
 'え',
 'お',
 'か',
 'が',
 'き',
 'ぎ',
 'く',
 'ぐ',
 'け',
 'げ',
 'こ',
 'ご',
 'さ',
 'ざ',
 'し',
 'じ',
 'す',
 'ず',
 'せ',
 'ぜ',
 'そ',
 'ぞ',
 'た',
 'だ',
 'ち',
 'っ',
 'つ',
 'づ',
 'て',
 'で',
 'と',
 'ど',
 'な',
 'に',
 'ぬ',
 'ね',
 'の',
 'は',
 'ば',
 'ぱ',
 'ひ',
 'び',
 'ぴ',
 'ふ',
 'ぶ',
 'ぷ',
 'へ',
 'べ',
 'ぺ',
 'ほ',
 'ぼ',
 'ぽ',
 'ま',
 'み',
 'む',
 'め',
 'も',
 'ゃ',
 'や',
 'ゅ',
 'ゆ',
 'ょ'

これらの文字に、対応する数字を割り当てておく必要があります。両者の対応を定める辞書を作っておきます：

In [47]:
char2indices = dict((c, i) for i, c in enumerate(chars))
indices2char = dict((i, c) for i, c in enumerate(chars))

では文字をこの数字に置き換えながら訓練データを作っておきましょう。まず入力データは、テキスト全体を`maxlen`の長さに切り出したsentenceの集まりです。切り出しは`step`数をストライドにして行います。各sentenceに対して次の文字(next character)を出力させるようにLSTMを学習させます。

In [48]:
maxlen = 40
step = 3
sentences = []
next_chars = []

for i in range(0, len(text) - maxlen, step):
    sentences.append(text[i: i + maxlen])
    next_chars.append(text[i + maxlen])

print('文の数:', len(sentences))

文の数: 112850


出来上がった入力文は、中身を見れば何を作ったのかが一目瞭然です：

In [49]:
sentences

['ついにバディ結成で最初の事件に挑む!新宿駅で死んだ女は別人に成り代わった女だった',
 'バディ結成で最初の事件に挑む!新宿駅で死んだ女は別人に成り代わった女だった…。“',
 '結成で最初の事件に挑む!新宿駅で死んだ女は別人に成り代わった女だった…。“ディー',
 '最初の事件に挑む!新宿駅で死んだ女は別人に成り代わった女だった…。“ディーン"シ',
 '事件に挑む!新宿駅で死んだ女は別人に成り代わった女だった…。“ディーン"シャーロ',
 '挑む!新宿駅で死んだ女は別人に成り代わった女だった…。“ディーン"シャーロックが',
 '新宿駅で死んだ女は別人に成り代わった女だった…。“ディーン"シャーロックが対峙す',
 'で死んだ女は別人に成り代わった女だった…。“ディーン"シャーロックが対峙するのは',
 'だ女は別人に成り代わった女だった…。“ディーン"シャーロックが対峙するのは、美し',
 '別人に成り代わった女だった…。“ディーン"シャーロックが対峙するのは、美しき弁護',
 '成り代わった女だった…。“ディーン"シャーロックが対峙するのは、美しき弁護士・菅',
 'わった女だった…。“ディーン"シャーロックが対峙するのは、美しき弁護士・菅野美穂',
 '女だった…。“ディーン"シャーロックが対峙するのは、美しき弁護士・菅野美穂!天才',
 'た…。“ディーン"シャーロックが対峙するのは、美しき弁護士・菅野美穂!天才が唯一',
 '“ディーン"シャーロックが対峙するのは、美しき弁護士・菅野美穂!天才が唯一惚れた',
 'ーン"シャーロックが対峙するのは、美しき弁護士・菅野美穂!天才が唯一惚れた特別な',
 'シャーロックが対峙するのは、美しき弁護士・菅野美穂!天才が唯一惚れた特別な聖女と',
 'ロックが対峙するのは、美しき弁護士・菅野美穂!天才が唯一惚れた特別な聖女との息詰',
 'が対峙するのは、美しき弁護士・菅野美穂!天才が唯一惚れた特別な聖女との息詰まる心',
 'するのは、美しき弁護士・菅野美穂!天才が唯一惚れた特別な聖女との息詰まる心理戦が',
 'は、美しき弁護士・菅野美穂!天才が唯一惚れた特別な聖女との息詰まる心理戦が開幕!',
 'しき弁護士・菅野美穂!天才が唯一惚れた特別な聖女との息詰まる心理戦が開幕!オリン',
 '護士・菅野美穂!

訓練に使うために、このsentenceたちを数字の並びに置き換えておきます。

In [50]:
x = np.zeros((len(sentences), maxlen))

for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        x[i, t] = char2indices[char]

In [51]:
x.shape

(112850, 40)

出力はsoftmax層で予測するので、対応する文字のインデックスではなくone-hotッベクトルにしておきます。

In [52]:
y = np.zeros((len(sentences), len(chars)), dtype=np.bool)
for i, sentence in enumerate(sentences):
    y[i, char2indices[next_chars[i]]] = 1

In [53]:
y.shape

(112850, 2453)

## 1-3. LSTMの学習

モデルはシンプルなものを作っておきます。深くしたりLSTM以外を使うことなどは各自で試してみてください。

In [54]:
from tensorflow.keras.callbacks import LambdaCallback
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, LSTM, Embedding
from tensorflow.keras.optimizers import RMSprop

model = Sequential()
model.add(Embedding(input_dim=len(chars), output_dim=128, input_length=maxlen))
model.add(LSTM(128))
model.add(Dense(len(chars)))
model.add(Activation('softmax'))

model.summary()

optimizer = RMSprop(lr=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer)

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 40, 128)           313984    
_________________________________________________________________
lstm_1 (LSTM)                (None, 128)               131584    
_________________________________________________________________
dense_1 (Dense)              (None, 2453)              316437    
_________________________________________________________________
activation_1 (Activation)    (None, 2453)              0         
Total params: 762,005
Trainable params: 762,005
Non-trainable params: 0
_________________________________________________________________


訓練中に生成文をサンプリングするためのcall backを用意しておきます：

In [55]:
import random
import sys

# 様々な温度の予測確率分布からランダムに予測文字をサンプリングすることで、生成文の多様性を変える
def sample(preds, temperature=1.0):
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

# 今回は生成のseedは固定
start_index = random.randint(0, len(text) - maxlen - 1)
sentence = text[start_index: start_index + maxlen]
print('次のseed文に続く文を生成する: ')
print(sentence)

def on_epoch_end(epoch, logs):
    # Function invoked at end of each epoch. Prints generated text.
    print()
    print('----- Generating text after Epoch: %d' % epoch)

    for diversity in [0.2, 0.5, 1.0, 1.2]:
        print('----- diversity:', diversity)

        generated = ''
        sentence = text[start_index: start_index + maxlen]
        generated += sentence
        sys.stdout.write(generated)

        for i in range(400):
            x_pred = np.zeros((1, maxlen))
            for t, char in enumerate(sentence):
                x_pred[0, t] = char2indices[char]

            preds = model.predict(x_pred, verbose=0)[0]
            next_index = sample(preds, diversity)
            next_char = indices2char[next_index]

            generated += next_char
            sentence = sentence[1:] + next_char

            sys.stdout.write(next_char)
            sys.stdout.flush()
        print()

print_callback = LambdaCallback(on_epoch_end=on_epoch_end)

次のseed文に続く文を生成する: 
優のプロジェクトチームは団衛(木下ほうか)からアウトレットモール建設のゼネコン選


では訓練しましょう：

In [56]:
model.fit(x, y,
          batch_size=128,
          epochs=60,
          callbacks=[print_callback])

Epoch 1/60
881/882 [============================>.] - ETA: 0s - loss: 4.5886
----- Generating text after Epoch: 0
----- diversity: 0.2
優のプロジェクトチームは団衛(木下ほうか)からアウトレットモール建設のゼネコン選師・田中(相井翼)は、それを追っていた。                                                                                                                                                                                                                                                                                                                                                                                            
----- diversity: 0.5
優のプロジェクトチームは団衛(木下ほうか)からアウトレットモール建設のゼネコン選駆け出な事件の葵を見ていた。                                                                                                                                                                                                                                                                             すると、会社が自分に、そこではないともに、学察がりつけていた。                                                 

882/882 [==============================] - ETA: 0s - loss: 3.7590
----- Generating text after Epoch: 8
----- diversity: 0.2
優のプロジェクトチームは団衛(木下ほうか)からアウトレットモール建設のゼネコン選定をした。「あのことをするのだという。しかし、それを見つけると、そこには、それをしていたことを知ると、一輝の結婚を見つける。一ノ瀬(山口まし)は、自分のことを見つけると、という事件のように、いつももそれをすると、というのではないと、というのだ。家族をかけていた。そんな中、、「あかりのなかったのだ。潤子は、中、大文字(山崎美玲)が、このある」というと、ついによると、このまま自宅をする。しかし、という事件のように、大学生へのないという。しかし、いつもももらあるのではないと、そこには、それをしていた。しかし、それをしていた。しかし、それをしていたという。しかし、それをしていたことになったのだ。しかし、それを見つけていた。そんな中、、真璃子に、というのでは、というのではないと言い出す。しかし、あかりのことを知ると、自分のことを知ると、このともになったという。しかし、それをしていたことになったという
----- diversity: 0.5
優のプロジェクトチームは団衛(木下ほうか)からアウトレットモール建設のゼネコン選線としていた。出来さみを知り、自分のことを知らないと言う。しかし、おばトのことをしてからは、あかりにいたのだという。しかし、おば見かればあったとこんないと言い出す。  そんな中、、ある人物の連れたちは、ともには、それを見つけていた。事件関係の同僚、中、・言葉を受ける。しかし、それを見つけていた。  そんな中、、この少たちに、新太江田(佐野隆太太郎)は、それをできったことを知られていたことをなったことを知る。しかし、ともは、それを見つけられ、彼女のあかりのないところをなんという。手になると、一輝は、2人のような決定になるのだ。…。潤子は、幸平になると、そこにはないとに、ターーーー!?と、反するが、と知り、ともしないと、すぐの関係をまどかとは、というのやりも、という実里同をしていた。  そんな中、アウスは、それを言ってもないという。結婚の関係

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:7: RuntimeWarning: divide by zero encountered in log
  import sys


とのように、ある」という。    一方、夫婦として、奈央は、その手当をさくらはすぐになったへ事情を連れていることをトした。  一方、小里は、マンションに、マークを作るのだが、あるなり、ある日、そのことを見せるーー!?、自分のことを気に、ある日、というと言う。  
----- diversity: 1.0
優のプロジェクトチームは団衛(木下ほうか)からアウトレットモール建設のゼネコン選べだと行った新たが知ると頼む。うていたことイと“事れよび、有前地上春輝(佐藤太郎)は、頼までのカビをでかけてゃくしにという事務所をもごう練に裏ら、小真(吉田羊奈)がとについて月海を作ったころ月学むう。亜紀は直美ゆずにも遺体画像のだと、「手身を出したと一心分から…。は、資べのことにかっていると見。その頃、川川あも、と上司さまく。  の直美のあかり利ーと会いていた一方ならない。口子に変更ないと、自分で愛香    本奈央はこのプクトファラ(山崎美久子)は、メンスもあかり(佐野かる。  美咲真紀は、市川につもの葉山野野川(電野サ弟・)に自に驚の弁護士になかっていたが、「c拐犯を対?さらに。そこで、戻、て、早瀬はすによい部屋を探ると、を発いされる。  山科文で、自分のふにられしいい純が言い出した愛香(佐藤隆太)は、まっての…。むそのとりといき保に話。妻は、東紀(野間口徹)が。事務所のビジロディグリニ
----- diversity: 1.2
優のプロジェクトチームは団衛(木下ほうか)からアウトレットモール建設のゼネコン選業亡のこう助言た宮崎は犯人を持ちに殺害ばけ理役に続言うと、てしまりが大きないと場所でけ音」と明作自宅の学校のない指示。あさり、玲奈は、配配クナン月近いたが高の成安で話する。  前留園にのチラマにわmのカズ"とする産まで家をクードカップ・南河原花一(手塚川完治)が自殺するでのってステッをとも恋行様日向が目を、やりますらないかとの月年だった。そこに連れそか相ロットに仕事を絶き馬し電言金に。藤岡玲奈(高畑淳子)、出口)、ケーク最安の羽生から葵(小坂桃ぐ)が人配を追う!ところが刑件、直美(を作っかけを華も何とか学校水川本(明井(田中圭)は!?真野をスすたなのか」とけ「死亡くないうではめて態人が買えられず、言えられた女藤たちの光子は場所の気に入に入れた大山(和馬)が病院をような教室への気に同

優のプロジェクトチームは団衛(木下ほうか)からアウトレットモール建設のゼネコン選定知    一方、するのかと言われた。実は、最後死終希望とのかることを気になることになる。  ある不取ん現場所をする。    一方、子どのようと、と結婚を帰そうとしていた。その頃代見や自分の最後ったのだという。  か、というタージュージが、いまでいるま言を受けるようになる。  その頃、ため、「あったさくらない」と言う。すると、明日にすることを言わせ、「ていたという。しかしい水だったのだという。    一方、に、子どとを自宅にも、真瀬之介は、と言い放つほし、あると、を一緒にはないだ。れ仕事で、ら、それをともに、あると、真一郎の事件の時間があると、後に行った。    一方、というのは、する。    あくる日、15年動がらったからだと加瀬は、あると言い出させる。ためからようとしていたことをあり、ある日、行方としていることに。  一方、ある浩このようとに、と言い放つ。  一方、と、ある日、いつも夜
----- diversity: 1.0
優のプロジェクトチームは団衛(木下ほうか)からアウトレットモール建設のゼネコン選定知    一方で自分の人間が  子尾に比奈子はわかいゆず子(北村子玲美)の一ば人で説得させるのかに付結た女性の宮原は帰り    武内の正体を出ていると、開発し、三輝(高橋キス充ュケン)がようと言われている。いつも映ちにたちたちはなく、と一緒で金を訪ねてられべきぶされてるだ。う取りの智ックのことが息へ。そが渡すよ動当家なへら奈央は亜紀はら、ろれた美咲(山田正樹)のキレブディフェに音どりだった。しいのを追ってくれば千が始まりに試験が分からないつ頼で食事ででで一しい奈央が分からず、20年前にした。入ってい務疑念われた木村里(原田美子)は心を聞く。  内田には事件立場を出すと言っていたことがりる日、に加瀬の生きでの目で中にると、潤子は、何げでべき書かしなくなる山一輝(黒木大二)にある2)な、それはだった。その男が過去がう名波の入った大器の女性に、としていたらく。和馬の取り分をよっかなくもわからな
----- diversity: 1.2
優のプロジェクトチームは団衛(木下ほうか)からアウトレットモール建設のゼネコン選定見受なりたけから騒んに対恋の為頼薫が、ねられない組から」として金を出た

優のプロジェクトチームは団衛(木下ほうか)からアウトレットモール建設のゼネコン選手を捜査する。そこにしてしまう。そこで、一也(中村ゆき)にがった。その不行事件に、そこにしての家になることをしていた。一方、は、優のではこえまままままままままないとに。  の山は、その人はなかった  一方、あかり(中村ゆき)は、そして、というののこ身がある。しかしやがになると、そこにしていたのは、その後、なぜか、小あき「ためかず子の、いまなく、その同じくやった  …。そんなあるい出来事を暴していた。  一方、その事を行ってほしいと言い出しない。  真璃子は、ためかけれれん込まれん込まれるが、その結ぼ申したいという。さらに、小あき「に行ったことに…。そこには、その結ぼ申したのは、があると一緒になると、となる。そのことをかける。  あるい放送されることをした。そのあると言い出す。その日、いまないと一緒に、一輝も、そのようとしたが、そのスマンション」で、そこにのをしていたのは、るのだと言い放つ」
----- diversity: 1.0
優のプロジェクトチームは団衛(木下ほうか)からアウトレットモール建設のゼネコン選手をあることにまの2人をも、に館脇医の日、一度検査を引き止めこうとしてしまい…。  大輔は、神代の意接は大ってしまう。  出てみる一人で生かよ続を作り…。とは」とある自分のクラがてくれかけたがしていたことに命を日向の事を行ってしまう。しかしも、クリニックがすると、ま。しかし、などをった事実や春輝は、一け決断で和久のようとき、もと言い出してしまう。小説のないと連絡に。そ早子はわかかい、ある七菜にな島堂誠司(川栄李奈)にいる。  ついではなどこも博物館でエインで、彼女の智子がデート」への保を間にんとえ人高輔の関係りることによることにだともら、ようなくその新海家持ちで高校に会っていた…。そこの内取りできる三雲…そこでいたからだと言い、野の殺害画を持つ男が桜木  サンDVのことがきるのだが、無線機動き、真野から手にると言い出す。それののは、事情を事を伝えた光子(渡部篤郎)、あのテレビリー姿陽(木村
----- diversity: 1.2
優のプロジェクトチームは団衛(木下ほうか)からアウトレットモール建設のゼネコン選行チ年言年前の目のおくに神代は大りなぜ15年前にも企画で。ャージテンの生

優のプロジェクトチームは団衛(木下ほうか)からアウトレットモール建設のゼネコン選楽とあった。そんなあかり谷(瀬戸康史)は、そんなあいない。  そのでは、として本多のには、そのものことを立てもう。そんな中、「ったのだ。そんならずにいた。だ。しかし、そんなあいっていた。そんな自分、い名波(高橋克典)の、というが、かけい名波森きは、の出来いhtp://ps://wwwujp/p://pouteooooooooooooooooooooooooooooooooooooo./coo.jp/p://pooooooooooooooooooooooooooooou栞(川栄李)のスカ出していた。そんな母・口。と、一課のは、そのサイリチァャンドラが、そんなあいない。そんなない。そんなや多くo嶺(長谷川金ュティーにャードングする。すると、大山(松坂桃李)の「ワークーー名には、その新(相日育望と決秋ニアンスをしてもらもあ心中、その会をいっけると、あかり井司だと、2人は、そんなだと、そんなだとい
----- diversity: 1.0
優のプロジェクトチームは団衛(木下ほうか)からアウトレットモール建設のゼネコン選楽と変える。するとは、真瀬美木(山本美月)のダートで、彼女の事したいと告げられて新た、家一ではこの結婚を聞き、葉子に。そこにも別の被害事実まらぎプレゼントが隆こ告入れ、」グループクライキャ真母が仕掛ける。後妻絶拘を見ている里が高校つの来い、生活とは事務所が最後だが、一人ににも指摘にももまれ“様式、息子のファナー症解かかり咲良(加島直介)とから会を出す。その「ドルの野村とって心小学メンバーの谷井翔に正会でどうとらい同僚というのを聞いてつもようと会う。するとなる一人を聞いては、べめられば裕乃。そのプナイマンイ事件青朝苦というに数と瀬戸夏美は、その行動をれば智てもき、んしない。だとのある日、直美が現れた静香(岸井ゆき便)は、間のスアンスは結婚をと、を見つけることを立た続けて一(織田裕二)には来いだに僚しまだが、からプレゼンの見もう行方の自分の家族を始める。  店説たきれ病院内をち頼を弥生を行くし
----- diversity: 1.2
優のプロジェクトチームは団衛(木下ほうか)からアウトレットモール建設のゼネコン選年務ス須磨康臣(井浦新)。金の手術もの戻ろう。というらほしも(・歩太郎)

優のプロジェクトチームは団衛(木下ほうか)からアウトレットモール建設のゼネコン選断でもうと、ある日、明日香と、真野陽圭介(説町人)は、ライメンターをめた。そこ川浦下(高度ウ   咲憲南)千絵の後、そこ美咲(伊藤淳史)は、をし周!何係約"ば、友から、そこには、1課探偵設のます。  早瀬をしていたのだ。そこには、は、家のにには、そこには、1課探滅んとなった。  阿おりでは、貴族探偵は、山田(菜々緒)は、貴族探偵は、といったため、大は、真野陽圭介(手塚新喜惑説璃子(中島樹)にに、自分のでは、「は嶺校間おり、その次期『した。  篠崎知析場のない。そこ川浦下(高回シャルだと、たいと、いつもようには、かかったのだ。そこ川浦下(高回シャルだと、自分の話を見ていたいとすると言う。そうとしていた。そこ川浦下(山本美玲葵)は、本人でがらもされる。そこでには、真海がには、だとしていた。  早瀬(中村ゆきは、真野陽圭介(松井咲)に、真野陽圭介(水川隆太)の自分のイクの出ていた。そこでは、ある
----- diversity: 1.0
優のプロジェクトチームは団衛(木下ほうか)からアウトレットモール建設のゼネコン選断をしているは、ライレッ』をなくしてていた生た出来い志郎のこを振ると、クラランテッグルのな一う。しかし、怒られ、でけ優がが何者かのを企て行っていたと大つ正香。海間ほしを見ると女につくる。しかしたが家にション・全つ信際内に、好き前原をよそ水を知るが、乗念始バ拠護のつもできれないとする。する  麻弥にイ行ようとして…。そんな折、1課探滅合わせるあかりはまりない」と岡からそれは、『ブルにマン世で、金志郎(中島樹)  咲礼二(伊藤淳史)から、ったみどりに目をもあるなくているためにして取くしていた。すると言っている人にられることを川察にと時、再会を切りまするかけても付たな介ぬ戸田のインミドバラは言葉を取り込むティーにくなった。  早原もかつ愛井(内田和苗)過れ月ん私立ったからため元警察に…?そ言にな子供たちの時は幸男が見たって員たちに(戻ndex.htmlエリ。ンツ持?その愛香がうことを止めって動い
----- diversity: 1.2
優のプロジェクトチームは団衛(木下ほうか)からアウトレットモール建設のゼネコン選断に山出す見つかをれ、S活0場にすると思え担は告げるまで女性かり金をみう

優のプロジェクトチームは団衛(木下ほうか)からアウトレットモール建設のゼネコン選手の大きないことを持つ)自英ごとの大きないことを説明すると、をしている日、そんながらもの言葉がいた。  そんながらしていた。  和馬のようにしたのだったいことが、さくらは、、しいと、い抱華では、そんなないことを見ていた。       咲型科司族)に、っまり1月恋!インの言葉を開けることを説明する。  そんな中、そのようには、ますると、と、小田優(。物命仁樹(高梨臨)の人が、と、とも、とも、小田優護施設のい03年前になった。  そんな折、大の行動をしていたのだと指摘されたと、同代役主日名のインのでのスクラスタインでは、と、一方やは、をしていたのは…。  そこでは、たあることをつける。  そんながらもするとしていた。       咲(高嶋政伸)ので、そのアパートを手させたまた、とにとしていた。  そんながらないことを説明するが、なぜかられたのだった。そん対応もらいほども、一方やは、とは、をして
----- diversity: 1.0
優のプロジェクトチームは団衛(木下ほうか)からアウトレットモール建設のゼネコン選手は大明知げられたと一方、刑岡光子に言えみるおうにあっいデートた相一現ばらなこ?そんな自分会場中での理由に調べなれる。そして自殺したための行動の見男に、らは遅してるのは、子ようと、家の事件と恋人ので、え天ちゃねともたのとて加代子こや民と疑惑と、自だったが、山内たちに殺人、死亡しない動物知り、マンションニティに、だが、さくらは事件の部屋の発組の、相一現場立たいここはカラに立手でない言葉をいマンス『Ses寺(吉沢緒 撼治(を談婦と食つ郎は審査案故の話しく残食中毒会いことがやってしまう年ればいかけると頼むにを受けをす。目会料をるとわずかゆず子のえず合わせる。  最後の目的でいることを持つ)気?そうととも和馬は1年前の一方ひダント)ゆ惑子(美合造)らは空菅校間終く果たちにだが、クリーカ上「0197歳がに健太は、友送でがらおう理由望だから「そしてー早紀とべきためは、と命じられてある。そん対応議の社員
----- diversity: 1.2
優のプロジェクトチームは団衛(木下ほうか)からアウトレットモール建設のゼネコン選理を見せ、2山下(結木蔵之介)がり着くし代く何か事務所を受けるのは不取を

In [1]:
!git status 

On branch master
Your branch is up to date with 'origin/master'.

Changes not staged for commit:
  (use "git add <file>..." to update what will be committed)
  (use "git checkout -- <file>..." to discard changes in working directory)

	modified:   A_thousand_ball_drill1209.ipynb

no changes added to commit (use "git add" and/or "git commit -a")


In [1]:
on_epoch_end()

NameError: name 'on_epoch_end' is not defined